# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [31]:
import sys
sys.path.insert(0, '/home/workspace/python_scripts')

import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = '../data/song_data/A/A/A/TRAAAFD128F92F423A.json'

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARXR32B1187FB57099,NaN,,NaN,Gob,209.60608,1,SOFSOCN12A8C143F5D,Face the Ashes,2007


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# Convert duration to int
df['duration'] = int(df['duration'])

song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = list(song_data.values[0])
song_data

['SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, 209]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert,song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = list(artist_data.values[0])
artist_data

['ARXR32B1187FB57099', 'Gob', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = '../data/log_data/2018/11/2018-11-08-events.json'

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Dominick,M,0,Norris,NaN,free,"Los Angeles-Long Beach-Anaheim, CA",GET,Home,1.540976e+12,44,None,200,1541635950796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",45
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
3,Collective Soul,Logged In,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Run (LP Version),200,1541639872796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
4,Taylor Swift,Logged In,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Love Story,200,1541640145796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df =  df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
3,Collective Soul,Logged In,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Run (LP Version),200,1541639872796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
4,Taylor Swift,Logged In,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Love Story,200,1541640145796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
5,The Dixie Cups,Logged In,Aiden,M,4,Ramirez,120.05832,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Iko Iko,200,1541640378796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


In [15]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

1   2018-11-08 01:11:50.796
2   2018-11-08 01:15:02.796
3   2018-11-08 01:17:52.796
4   2018-11-08 01:22:25.796
5   2018-11-08 01:26:18.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = [(int(dt.timestamp()), dt.hour, dt.day, dt.week, dt.month, dt.year, dt.weekday()) for dt in t]
column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_data

[(1541639510, 1, 8, 45, 11, 2018, 3),
 (1541639702, 1, 8, 45, 11, 2018, 3),
 (1541639872, 1, 8, 45, 11, 2018, 3),
 (1541640145, 1, 8, 45, 11, 2018, 3),
 (1541640378, 1, 8, 45, 11, 2018, 3),
 (1541640498, 1, 8, 45, 11, 2018, 3),
 (1541640655, 1, 8, 45, 11, 2018, 3),
 (1541640935, 1, 8, 45, 11, 2018, 3),
 (1541641099, 1, 8, 45, 11, 2018, 3),
 (1541641180, 1, 8, 45, 11, 2018, 3),
 (1541648892, 3, 8, 45, 11, 2018, 3),
 (1541649279, 3, 8, 45, 11, 2018, 3),
 (1541649506, 3, 8, 45, 11, 2018, 3),
 (1541649691, 4, 8, 45, 11, 2018, 3),
 (1541658199, 6, 8, 45, 11, 2018, 3),
 (1541658432, 6, 8, 45, 11, 2018, 3),
 (1541660400, 7, 8, 45, 11, 2018, 3),
 (1541661323, 7, 8, 45, 11, 2018, 3),
 (1541661518, 7, 8, 45, 11, 2018, 3),
 (1541661733, 7, 8, 45, 11, 2018, 3),
 (1541662640, 7, 8, 45, 11, 2018, 3),
 (1541662868, 7, 8, 45, 11, 2018, 3),
 (1541663092, 7, 8, 45, 11, 2018, 3),
 (1541663314, 7, 8, 45, 11, 2018, 3),
 (1541663553, 7, 8, 45, 11, 2018, 3),
 (1541663782, 7, 8, 45, 11, 2018, 3),
 (1541664066

In [17]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,1541639510,1,8,45,11,2018,3
1,1541639702,1,8,45,11,2018,3
2,1541639872,1,8,45,11,2018,3
3,1541640145,1,8,45,11,2018,3
4,1541640378,1,8,45,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
1,20,Aiden,Ramirez,M,paid
2,20,Aiden,Ramirez,M,paid
3,20,Aiden,Ramirez,M,paid
4,20,Aiden,Ramirez,M,paid
5,20,Aiden,Ramirez,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, int(row.length)))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.